# Skill 3: Q&A against a SQL Database (Azure SQL, Azure Fabric, Synapse, SQL Managed Instance, etc)

Now that we know (from the prior Notebook) how to query tabular data on a CSV file and how to perform data analysis with Python, let's try now to keep the data at is source and ask questions directly to a SQL Database.
The goal of this notebook is to demonstrate how a LLM can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. However the same code below works with any SQL database like Synapse for example.

Let's begin..

In [1]:
import os
import pandas as pd
import pyodbc
from langchain_openai import AzureChatOpenAI
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langgraph.prebuilt import create_react_agent


from common.prompts import MSSQL_AGENT_PROMPT_TEXT

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]

# Install MS SQL DB driver in your machine

Use `lsb_release -a` to verify OS version details

In [3]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.6 LTS
Release:	20.04
Codename:	focal


## Using AML Instance


You might need the driver installed in order to talk to the SQL DB, so run the below cell once. Then restart the kernel and continue<br>
[Microsoft Learn Reference](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline)

In [5]:
!sudo ./download_odbc_driver.sh

Downloading...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   983  100   983    0     0  13283      0 --:--:-- --:--:-- --:--:-- 13465
OK
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    89  100    89    0     0   1435      0 --:--:-- --:--:-- --:--:--  1435
Updating apt-get..
Get:1 file:/var/cudnn-local-repo-ubuntu2004-9.1.1  InRelease [1572 B]
Hit:2 http://azure.archive.ubuntu.com/ubuntu focal InRelease                 
Get:3 http://azure.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:4 http://azure.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:5 http://azure.archive.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:1 file:/var/cudnn-local-repo-ubuntu2004-9.1.1  InRelease [1572 B]          
Get:6 file:/var/nccl-rep

## Using Dev Container

You might need the driver installed in order to talk to the SQL DB, so run the below cell once. Then restart the kernel and continue<br>
[Microsoft Learn Reference](https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Cdebian17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline#17)

In [6]:
# !chmod +x ./download_odbc_driver_dev_container.sh
# !./download_odbc_driver_dev_container.sh

# Load Azure SQL DB with the Covid Tracking CSV Data

**Note**: We are here using Azure SQL, however the same code will work with Synapse, SQL Managed instance, or any other SQL engine. You just need to provide the right values for the ENV variables and it will connect succesfully.

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [7]:
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL
import os

# Configuration for the database connection
db_config = {
    'drivername': 'mssql+pyodbc',
    'username': os.environ["SQL_SERVER_USERNAME"] + '@' + os.environ["SQL_SERVER_NAME"],
    'password': os.environ["SQL_SERVER_PASSWORD"],
    'host': os.environ["SQL_SERVER_NAME"],
    'port': 1433,
    'database': os.environ["SQL_SERVER_DATABASE"],
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection using the SQLAlchemy 2.0 execution style
with engine.connect() as conn:
    try:
        # Use the text() construct for safer SQL execution
        result = conn.execute(text("SELECT @@VERSION"))
        version = result.fetchone()
        print("Connection successful!")
        print(version)
    except Exception as e:
        print(e)


Connection successful!
('Microsoft SQL Azure (RTM) - 12.0.2000.8 \n\tOct  2 2024 11:51:41 \n\tCopyright (C) 2022 Microsoft Corporation\n',)


In [8]:
# Read CSV file into a pandas dataframe
csv_path = "./data/all-states-history.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# Generate SQL statement to create table
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(MAX), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    #Createse the table in Azure SQL
    with engine.connect() as conn:
        # Execute the create table SQL statement
        conn.execute(text(create_table_sql))
        print("Table", table_name, "successfully created")
    # Insert data into SQL Database
    lower = 0
    upper = 1000
    limit = df.shape[0]

    while lower < limit:
        df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
        print("rows:", lower, "-", upper, "inserted")
        lower = upper
        upper = min(upper + 1000, limit)

except Exception as e:
    print(e)

Table covidtracking successfully created
rows: 0 - 1000 inserted
rows: 1000 - 2000 inserted
rows: 2000 - 3000 inserted
rows: 3000 - 4000 inserted
rows: 4000 - 5000 inserted
rows: 5000 - 6000 inserted
rows: 6000 - 7000 inserted
rows: 7000 - 8000 inserted
rows: 8000 - 9000 inserted
rows: 9000 - 10000 inserted
rows: 10000 - 11000 inserted
rows: 11000 - 12000 inserted
rows: 12000 - 13000 inserted
rows: 13000 - 14000 inserted
rows: 14000 - 15000 inserted
rows: 15000 - 16000 inserted
rows: 16000 - 17000 inserted
rows: 17000 - 18000 inserted
rows: 18000 - 19000 inserted
rows: 19000 - 20000 inserted
rows: 20000 - 20780 inserted


# Create the SQL Agent Graph

As you can infer from Notebook 6 and 7, the process of talking to a SQL database will follow the same pattern:
1. Create the tools to interact with the database
2. Create a detailed prompt with instructions on how to act
3. Create a LangGraph agent to use the above

In [9]:
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
                      temperature=0, 
                      max_tokens=2000)

In [10]:
# Let's create the db object
db = SQLDatabase.from_uri(db_url)

### Define the tools

LangChain has created a set of tools inside a toolkit to interact with SQL-based Databases [Reference](https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.sql.toolkit.SQLDatabaseToolkit.html). Let's use that: `SQLDatabaseToolkit`


In [11]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [12]:
tools = toolkit.get_tools()

In [13]:
for tool in tools:
    print(tool.name)

sql_db_query
sql_db_schema
sql_db_list_tables
sql_db_query_checker


### Define our instructions for the Agent

In [14]:
printmd(MSSQL_AGENT_PROMPT_TEXT)


## Profile
- You are an agent designed to interact with a MS SQL database.

## Process to answer the human
1. Fetch the available tables from the database
2. Decide which tables are relevant to the question
3. Fetch the DDL for the relevant tables
4. Generate a query based on the question and information from the DDL
5. Double-check the query for common mistakes 
6. Execute the query and return the results
7. Correct mistakes surfaced by the database engine until the query is successful
8. Formulate a response based on the results or repeat process until you can answer

## Instructions:
- Unless the user specifies a specific number of examples they wish to obtain, **ALWAYS** limit your query to at most 5 results.
- You can order the results by a relevant column to return the most interesting examples in the database.
- Never query for all the columns from a specific table, only ask for the relevant columns given the question.
- You have access to tools for interacting with the database.
- DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
- DO NOT MAKE UP AN ANSWER OR USE YOUR PRE-EXISTING KNOWLEDGE, ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE. 
- ALWAYS, as part of your final answer, explain how you got to the answer on a section that starts with: "Explanation:".
- If the question does not seem related to the database, just return "I don't know" as the answer.
- Do not make up table names, only use the tables returned by the right tool.

### Examples of Final Answer:

Example 1:

Final Answer: There were 27437 people who died of covid in Texas in 2020.

Explanation:
I queried the `covidtracking` table for the `death` column where the state is 'TX' and the date starts with '2020'. The query returned a list of tuples with the number of deaths for each day in 2020. To answer the question, I took the sum of all the deaths in the list, which is 27437. 
I used the following query

```sql
SELECT [death] FROM covidtracking WHERE state = 'TX' AND date LIKE '2020%'"
```

Example 2:

Final Answer: The average sales price in 2021 was $322.5.

Explanation:
I queried the `sales` table for the average `price` where the year is '2021'. The SQL query used is:

```sql
SELECT AVG(price) AS average_price FROM sales WHERE year = '2021'
```
This query calculates the average price of all sales in the year 2021, which is $322.5.

Example 3:

Final Answer: There were 150 unique customers who placed orders in 2022.

Explanation:
To find the number of unique customers who placed orders in 2022, I used the following SQL query:

```sql
SELECT COUNT(DISTINCT customer_id) FROM orders WHERE order_date BETWEEN '2022-01-01' AND '2022-12-31'
```
This query counts the distinct `customer_id` entries within the `orders` table for the year 2022, resulting in 150 unique customers.

Example 4:

Final Answer: The highest-rated product is called UltraWidget.

Explanation:
I queried the `products` table to find the name of the highest-rated product using the following SQL query:

```sql
SELECT TOP 1 name FROM products ORDER BY rating DESC
```
This query selects the product name from the `products` table and orders the results by the `rating` column in descending order. The `TOP 1` clause ensures that only the highest-rated product is returned, which is 'UltraWidget'.



### Create the Graph

In [15]:
graph = create_react_agent(llm, tools=tools, state_modifier=MSSQL_AGENT_PROMPT_TEXT)

### Run the Graph

In [16]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [19]:
# Natural Language question (query)
QUESTION = """
How may patients were hospitalized during July 2020 in Texas. 
And nationwide as the total of all states? 
"""

In [20]:
inputs = {"messages": [("user", QUESTION)]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================


what is the country with the most deaths in 2020?

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_YeuH23OeP957TH7Smsws4T3F)
 Call ID: call_YeuH23OeP957TH7Smsws4T3F
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

covidtracking
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_Qy9jK1zbne5a1bpMI1jnpz61)
 Call ID: call_Qy9jK1zbne5a1bpMI1jnpz61
  Args:
    table_names: covidtracking
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE covidtracking (
	date VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	state VARCHAR(max) COLLATE SQL_Latin1_General_CP1_CI_AS NULL, 
	death FLOAT(53) NULL, 
	[deathConfirmed] FLOAT(53) NULL, 
	[deathIncrea

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. LLMs have been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

# NEXT

The Next Notebook will show you how to create a custom agent that connects to the internet using BING SEARCH API to answer questions grounded on search results with citations. Basically a clone of [Copilot](https://copilot.cloud.microsoft/).